In [72]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

EPS = 2 / 255.0
LR = 0.1

In [73]:
def preprocess_image(image,dim=(224,224)):
    # swap color channels, resize the input image, and add a batch dimension
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, dim)
    image = np.expand_dims(image, axis=0)
    return image

In [74]:
def get_class_idx(class_name):
  import urllib.request
  import json

  url = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
  response = urllib.request.urlopen(url)
  class_names = json.loads(response.read())
  for i in range(len(class_names)):
    if class_names[i] == class_name:
      return i

In [75]:
def predict(model,image,decode_predictions):
  predictions = model.predict(image)
  predictions = decode_predictions(predictions, top=3)[0]
  label = predictions[0][1]
  confidence = predictions[0][2] * 100
  print("[INFO] label: {} confidence: {:.2f}%".format(label, confidence))
  return label.replace('_', ' ')


In [76]:
def clip_eps(tensor, eps):
    return tf.clip_by_value(tensor, clip_value_min=-eps, clip_value_max=eps)

In [77]:
def generate_adversaries(model, baseImage, delta, classIdx, steps=50):
    optimizer = Adam(learning_rate=LR)
    sccLoss = SparseCategoricalCrossentropy() 
    for step in range(0, steps):
        with tf.GradientTape() as tape:
            tape.watch(delta)
            adversary = preprocess_input(baseImage + delta)
            predictions = model(adversary, training=False)
            loss = -sccLoss(tf.convert_to_tensor([classIdx]), predictions)
            if step % 5 == 0:
                print("step: {}, loss: {}...".format(step, loss.numpy()))

        gradients = tape.gradient(loss, delta)
        optimizer.apply_gradients([(gradients, delta)])
        delta.assign_add(clip_eps(delta, eps=EPS))
    return delta

In [78]:
def attack(image,classIdx,output):
  baseImage = tf.constant(image, dtype=tf.float32)
  delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)

  print("[INFO] generating perturbation...")
  deltaUpdated = generate_adversaries(model, baseImage, delta, classIdx)
  print("[INFO] creating adversarial example...")
  adverImage = (baseImage + deltaUpdated).numpy().squeeze()
  adverImage = np.clip(adverImage, 0, 255).astype("uint8")
  adverImage = cv2.cvtColor(adverImage, cv2.COLOR_RGB2BGR)
  cv2.imwrite(output, adverImage)

  # save the noise vector as an image
  noiseImage = deltaUpdated.numpy().squeeze()
  noiseImage = np.clip((noiseImage + 0.5) * 255, 0, 255).astype("uint8")
  noiseImage = cv2.cvtColor(noiseImage, cv2.COLOR_RGB2BGR)
  noiseOutput = os.path.splitext(output)[0] + '_noise.jpg'
  cv2.imwrite(noiseOutput, noiseImage)
  return baseImage + deltaUpdated

In [79]:
def run(model,input,output,dim=(224,224)):
  print("[INFO] running inference on the original image...")
  image = cv2.imread(input)
  image = preprocess_image(image,dim)
  preprocessedImage = preprocess_input(image)
  label = predict(model,preprocessedImage,decode_predictions)
  classIdx = get_class_idx(label)
  print()

  adverImage = attack(image,classIdx,output)
  print("[INFO] running inference on the adversarial example...")
  preprocessedImage = preprocess_input(adverImage)
  predict(model,preprocessedImage,decode_predictions)

In [80]:
!wget https://raw.githubusercontent.com/utkuozbulak/pytorch-cnn-adversarial-attacks/master/input_images/apple.JPEG

--2023-05-07 15:49:25--  https://raw.githubusercontent.com/utkuozbulak/pytorch-cnn-adversarial-attacks/master/input_images/apple.JPEG
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5160 (5.0K) [image/jpeg]
Saving to: ‘apple.JPEG.2’

apple.JPEG.2        100%[===================>]   5.04K  --.-KB/s    in 0s      

2023-05-07 15:49:25 (76.8 MB/s) - ‘apple.JPEG.2’ saved [5160/5160]



In [81]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions,preprocess_input

print("[INFO] loading pre-trained ResNet50 model...")
model = ResNet50(weights="imagenet")
run(model,input = '/content/apple.JPEG',output = '/content/dogncat_adv_resnet.png')

[INFO] loading pre-trained ResNet50 model...
[INFO] running inference on the original image...
1/1 [==============================] - 1s 883ms/step
[INFO] label: Granny_Smith confidence: 87.95%

[INFO] generating perturbation...
step: 0, loss: -0.1284576654434204...
step: 5, loss: -6.641036033630371...
step: 10, loss: -15.304479598999023...
step: 15, loss: -23.006061553955078...
step: 20, loss: -31.427316665649414...
step: 25, loss: -40.05393981933594...
step: 30, loss: -47.87111282348633...
step: 35, loss: -56.047882080078125...
step: 40, loss: -63.69990158081055...
step: 45, loss: -70.79307556152344...
[INFO] creating adversarial example...
[INFO] running inference on the adversarial example...
1/1 [==============================] - 0s 32ms/step
[INFO] label: saltshaker confidence: 100.00%


In [82]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import decode_predictions,preprocess_input

print("[INFO] loading pre-trained VGG16 model...")
model = VGG16(weights="imagenet")
run(model,input='/content/apple.JPEG',output = '/content/apple_adv_vgg16.png')

[INFO] loading pre-trained VGG16 model...
[INFO] running inference on the original image...
1/1 [==============================] - 0s 139ms/step
[INFO] label: Granny_Smith confidence: 79.19%

[INFO] generating perturbation...
step: 0, loss: -0.23330824077129364...
step: 5, loss: -1.8835047483444214...
step: 10, loss: -4.744152069091797...
step: 15, loss: -8.235982894897461...
step: 20, loss: -12.371561050415039...
step: 25, loss: -16.82699203491211...
step: 30, loss: -21.407878875732422...
step: 35, loss: -26.699193954467773...
step: 40, loss: -32.87017059326172...
step: 45, loss: -39.921207427978516...
[INFO] creating adversarial example...
[INFO] running inference on the adversarial example...
1/1 [==============================] - 0s 20ms/step
[INFO] label: whistle confidence: 100.00%
